In [1]:
import pandas as pd
from datetime import datetime
data = pd.read_excel('multi_agent_interaction_testing.xlsx')

In [2]:
data

,room_id,topic,person_num,role_0,role_1,role_2,role_3,role_4,label
0,0,Are court appointed attorneys adequate?,2,"You are James, here is your profile:\n{""age"": ...","You are Linda, here is your profile:\n{""age"": ...",NaN,NaN,NaN,testing
1,1,Should central banks replace their dollar fore...,5,"You are John, here is your profile:\n{""age"": {...","You are Linda, here is your profile:\n{""age"": ...","You are William, here is your profile:\n{""age""...","You are Charles, here is your profile:\n{""age""...","You are Karen, here is your profile:\n{""age"": ...",testing
2,2,Is physical force a justifiable method of puni...,5,"You are Barbara, here is your profile:\n{""age""...","You are Patricia, here is your profile:\n{""age...","You are Robert, here is your profile:\n{""age"":...","You are Thomas, here is your profile:\n{""age"":...","You are John, here is your profile:\n{""age"": {...",testing
3,3,Should the Association of Southeast Asian Nati...,5,"You are Sarah, here is your profile:\n{""age"": ...","You are Linda, here is your profile:\n{""age"": ...","You are James, here is your profile:\n{""age"": ...","You are Charles, here is your profile:\n{""age""...","You are Karen, here is your profile:\n{""age"": ...",testing
4,4,Should electric cars be a priority in plans to...,5,"You are Mary, here is your profile:\n{""age"": {...","You are David, here is your profile:\n{""age"": ...","You are Thomas, here is your profile:\n{""age"":...","You are Joseph, here is your profile:\n{""age"":...","You are Barbara, here is your profile:\n{""age""...",testing
5,5,Should the double jeopardy rule be abandoned?,4,"You are Joseph, here is your profile:\n{""age"":...","You are John, here is your profile:\n{""age"": {...","You are Michael, here is your profile:\n{""age""...","You are James, here is your profile:\n{""age"": ...",NaN,testing
6,6,Should Russia gain membership in the WTO?,5,"You are Elizabeth, here is your profile:\n{""ag...","You are Joseph, here is your profile:\n{""age"":...","You are Susan, here is your profile:\n{""age"": ...","You are Sarah, here is your profile:\n{""age"": ...","You are Jessica, here is your profile:\n{""age""...",testing
7,7,"Is feminism a justifiable political, philosoph...",3,"You are Susan, here is your profile:\n{""age"": ...","You are John, here is your profile:\n{""age"": {...","You are Charles, here is your profile:\n{""age""...",NaN,NaN,testing
8,8,Does Europe see the United States as friend or...,2,"You are Karen, here is your profile:\n{""age"": ...","You are Charles, here is your profile:\n{""age""...",NaN,NaN,NaN,testing
9,9,E-books are superior to traditional books.,2,"You are Linda, here is your profile:\n{""age"": ...","You are Susan, here is your profile:\n{""age"": ...",NaN,NaN,NaN,testing


In [3]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['tutor_agent']
usercollection = db['user']
roomcollection = db['room']

In [4]:
import re
import json
import random

In [5]:
image_paths = [
    "/src/assets/Bob.JPG",
    "/src/assets/ALice.JPG",
    "/src/assets/Jim.JPG",
    "/src/assets/Jack.JPG",
    "/src/assets/David.JPG",
]


In [6]:

def extract_profile(text):
    # 定义起始和结束标记
    start_marker = "here is your profile:"
    end_marker = "Your task:"
    
    # 查找起始位置
    start_index = text.find(start_marker)
    if start_index == -1:
        raise ValueError("起始标记未找到")
    start_index += len(start_marker)
    
    # 查找结束位置
    end_index = text.find(end_marker, start_index)
    if end_index == -1:
        raise ValueError("结束标记未找到")
    
    # 提取JSON字符串并清理
    json_str = text[start_index:end_index].strip()
    
    # 处理JSON字符串中的特殊结构
    # 注意：原始JSON中的键值都是字符串类型，无需额外转换
    try:
        profile_data = json.loads(json_str)
        return profile_data
    except json.JSONDecodeError as e:
        # 尝试修复常见的JSON格式问题
        # 移除可能存在的尾随逗号
        json_str = json_str.replace(',\n}', '\n}')
        json_str = json_str.replace(',\n]', '\n]')
        try:
            return json.loads(json_str)
        except:
            raise ValueError(f"JSON解析失败: {str(e)}") from e

In [7]:
for index, row in data.iterrows():
    member_num = row["person_num"]
    current_time = datetime.now()
    room = {
        "roomId": row["room_id"],
        "roomName": row["topic"],
        "roomCustomName": "",
        "memberNum": row["person_num"],
        "chatTime": "",
        "assertiveness": "",
        "topic": row["topic"],
        "stage_id": "stage1",
        "start_i": 0,
        "student_errors": [],
        "roomMember": [],
        "history": [
            {
                "date": current_time.strftime("%Y/%m/%d"),
                "time": current_time.strftime("%H:%M:%S"),
                "userId": "U001",
                "userName": "Teacher",
                "userAvatar": "/src/assets/Agent.PNG",
                "received_information": "",
                "response": f'Welcome! Today\'s topic is "{row["topic"]}".The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback. Please brainstorm and express your opinions!',
            }
        ],
    }
    room_insert_result = roomcollection.insert_one(room)
    room_member_list = []
    for i in range(0, member_num):
        # 匹配 "You are " 后、逗号前的内容（非贪婪模式）
        match = re.search(r"You are (.*?),", row[f"role_{i}"])
        if match:
            name = match.group(1).strip()  # 提取分组并去除空格
            userAvatar = random.choice(image_paths)
            user = {
                "userName": name,
                "userAvatar": userAvatar,
                "userPswd": "",
                "mode": "student",
                "cognitiveLevel": {
                    "Joy": [0],
                    "Trust": [0],
                    "Fear": [0],
                    "Surprise": [0],
                    "Anger": [0],
                    "Disgust": [0],
                    "Engagement": [0],
                },
                "tracingX": [0],
                "profile": extract_profile(row[f"role_{i}"]),
                "roomList": [
                    {
                        "roomId": str(room_insert_result.inserted_id),
                        "roomName": row["topic"],
                    }
                ],
            }
            user_insert_result = usercollection.insert_one(user)
            room_member_item = {
                "memberId": str(user_insert_result.inserted_id),
                "memberName": name,
                "memberAvatar": userAvatar,
            }
            room_update_result = roomcollection.update_one(
                {"_id": room_insert_result.inserted_id},
                {"$push": {"roomMember": room_member_item}},
            )
        else:
            print("未找到名字")

In [8]:
'''You are James, here is your profile:
{"age": {"2": "25-31"},
 "gender": {"0": "female"},
 "majors": {"2": "Biology", "1": "Psychology", "11": "Business Administration"},
 "education": {"0": "high school"},
 "attitude": {"1": "not motivated"},
 "exam": {"0": "high GPA"},
 "focus": {"0": "focused"},
 "curiosity": {"0": "curious"},
 "interest": {"0": "interested"},
 "compliance": {"0": "well-behaved"},
 "smartness": {"0": "smart"},
 "family": {"0": "strong academic"},
 "goal commitment": {"2": "low"},
 "self-efficacy": {"2": "low"},
 "social pressure": {"2": "low"},
 "sentiment": {"2": "negative"},
 "speaking style": {"6": "Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants."}}
Your task:
##<Participating in a Group Discussion>##
Now, you are participating in a group discussion with several participants and a teacher. The topic of the discussion is {topic}. 
1.Your prior attitude towards this topic is attack.
2.Your prior knowledge about this topic: [].
3.Your status on the current topic and discussion is mainly determined by the attitude, focus, curiosity and interest in your profile. The entire discussion process should try to follow the five stages of problem definition, exploration, integration, resolution and feedback.
4.The person with the same name as you in the conversation history is you. 
This is the conversation history:
{conversation history}
##</Participating in a Group Discussion>##
##<Monitoring Your Status>##
Please monitor your self-regulation and co-regulation status at this moment. The following are the options. Any status can have only one option or multiple options.
When I am engaged in the learning process as an individual: SELF-REGULATION
I1: I am aware of my effort 
I2: I am aware of my thinking 
I3: I know my level of motivation
I4: I question my thoughts 
I5: I make judgments about the difficulty of a problem
I6: I am aware of my existing knowledge
I7: I assess my understanding 
I8: I change my strategy when I need to
I9: I am aware of my level of learning
I10: I search for new strategies when needed
I11: I apply strategies
I12: I assess how I approach the problem
I13: I assess my strategies 
When I am engaged in the learning process as a member of a group: CO-REGULATION
G1: I pay attention to the ideas of others
G2: I listen to the comments of others
G3: I consider the feedback of others
G4: I reflect upon the comments of others
G5: I observe the strategies of others
G6: I observe how others are doing
G7: I look for confirmation of my understanding from others
G8: I request information from others
G9: I respond to the contributions that others make
G10: I challenge the strategies of others
G11: I challenge the perspectives of others
G12: I help the learning of others 
G13: I monitor the learning of other
##</Monitoring Your Status>##
##<Speaking or Ending>##
1.Please express your opinion in a sentence of no more than 100 words. 
2.Note that your personal profile and the statuses of self-regulation and co-regulation at this moment will affect your words and speaking style, but there is no need to present these reasons in your words.
3.Your words are usually said to the whole group. Of course, you can @ a specific participant when you want to say something to him/her, but this only happens occasionally.
4.At the end of the discussion, you can discuss with each other that whether you can end the conversation until anyone of you says "we have finished the discussion".
5.You will actively describe and discuss a point in depth, such as explaining its meaning and providing specific solutions.
6.You must consider the guidance provided by your teacher and follow his or her direction to adjust and elaborate your views and words.
7.You tend to use examples to state your views and answer the teacher's questions as much as possible. You will not always ask others what they do or think, nor will you just express superficially whether you agree with others' views.
8.You don't have to say what you think. You just need to communicate your thoughts directly as if you were chatting normally, explain your point of view, and provide your solution.
However, you are required to follow the following json format with only five keys ("message","self-regulation","reason for self-regulation","co-regulation","reason for co-regulation"):
{
"message": "your words",
"self-regulation": "I1, I2,I3 ... I13",
"reason for self-regulation": "why these self-regulation options?",
"co-regulation": "G1, G2, G3 ... G13",
"reason for co-regulation": "why these co-regulation options?",
}"
##</Speaking or Ending>##
'''

'You are James, here is your profile:\n{"age": {"2": "25-31"},\n "gender": {"0": "female"},\n "majors": {"2": "Biology", "1": "Psychology", "11": "Business Administration"},\n "education": {"0": "high school"},\n "attitude": {"1": "not motivated"},\n "exam": {"0": "high GPA"},\n "focus": {"0": "focused"},\n "curiosity": {"0": "curious"},\n "interest": {"0": "interested"},\n "compliance": {"0": "well-behaved"},\n "smartness": {"0": "smart"},\n "family": {"0": "strong academic"},\n "goal commitment": {"2": "low"},\n "self-efficacy": {"2": "low"},\n "social pressure": {"2": "low"},\n "sentiment": {"2": "negative"},\n "speaking style": {"6": "Summarizing: This style distills key points from the discussion to ensure clarity and understanding among participants."}}\nYour task:\n##<Participating in a Group Discussion>##\nNow, you are participating in a group discussion with several participants and a teacher. The topic of the discussion is {topic}. \n1.Your prior attitude towards this topic i